# Załącznik nr 2: Krzyżowy dobór najlepszych parametrów klasyfikatorów

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import time
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

# Collab

In [2]:
import multiprocessing
n_cpus = multiprocessing.cpu_count()
n_cpus

12

In [3]:
!cat /proc/cpuinfo

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


# BUDOWANIE MODELI

In [3]:
dfj = pd.read_excel("2022_06_24mgrpythonPY.xlsx")

In [19]:
# Create correlation matrix
corr_matrix = dfj.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.50)]
upper

,ZAG_LUDNOŚCI,GDP_NA_OS,DNI POWSZEDNIE_WEEKEND,SPECJALNA_BUDOWA_OBIEKTU,CZY_BUDYNEK_JEST_NORMALNIE_ZAJĘTY,LICZBA_CZUJNIKÓW_DYMU,PÓŹNE_WYWOŁANIE,PRZYPADKOWE_CZY_CELOWE,CZY_SĄSIEDNI_OBIEKT_ZOSTAŁ_DOTKNIĘTY_POŻAREM,KOD_POJAZDÓW,KOD_PERSONELU,KOD_CZAS_INCYDENTU,POSZKODOWANI_MARTWI,URATOWANI,KOD_EWAKUOWANI,KOD_OBSZAR_USZKODZONY_PRZEZ_POŻAR,KOD_OBSZAR USZKODZONY PRZEZ POŻAR_ORAZ_WODĘ,SĄSIEDNI_OBIEKT_USZKODZONY
ZAG_LUDNOŚCI,NaN,0.192128,0.000780,0.014880,0.006123,0.015935,0.011962,0.002046,0.008903,0.001644,0.031888,0.022097,0.012982,0.004780,0.001890,0.027391,0.007469,0.007180
GDP_NA_OS,NaN,NaN,0.003053,0.016876,0.033236,0.047514,0.067124,0.059825,0.024105,0.008024,0.043461,0.037612,0.047097,0.013969,0.023160,0.104431,0.092682,0.020871
DNI POWSZEDNIE_WEEKEND,NaN,NaN,NaN,0.003344,0.001655,0.004254,0.023074,0.028609,0.008131,0.009283,0.003648,0.012117,0.010982,0.020036,0.005173,0.005824,0.005183,0.006991
SPECJALNA_BUDOWA_OBIEKTU,NaN,NaN,NaN,NaN,0.043892,0.038971,0.008959,0.008418,0.064821,0.023150,0.005100,0.040048,0.011757,0.002999,0.014683,0.116589,0.071325,0.066783
CZY_BUDYNEK_JEST_NORMALNIE_ZAJĘTY,NaN,NaN,NaN,NaN,NaN,0.168838,0.008612,0.253785,0.078908,0.013987,0.002125,0.040763,0.050373,0.026715,0.020254,0.147694,0.104350,0.080433
LICZBA_CZUJNIKÓW_DYMU,NaN,NaN,NaN,NaN,NaN,NaN,0.017896,0.116330,0.080699,0.003653,0.012833,0.013442,0.062244,0.038953,0.026458,0.083631,0.009984,0.080234
PÓŹNE_WYWOŁANIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042068,0.033114,0.100665,0.022641,0.108108,0.021714,0.029714,0.012383,0.039559,0.051160,0.027836
PRZYPADKOWE_CZY_CELOWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.091178,0.037872,0.001067,0.086634,0.002225,0.057666,0.009617,0.150449,0.110919,0.094163
CZY_SĄSIEDNI_OBIEKT_ZOSTAŁ_DOTKNIĘTY_POŻAREM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044336,0.003274,0.091615,0.036145,0.044077,0.031995,0.302498,0.225562,0.887150
KOD_POJAZDÓW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.237850,0.923735,0.041491,0.039678,0.019454,0.097268,0.089560,0.049911


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

dfj = pd.read_excel('/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/2022_06_24mgrpythonPY.xlsx')

Mounted at /content/drive/


In [16]:
data_type= dfj.dtypes
data_type

ZAG_LUDNOŚCI                                       int64
GDP_NA_OS                                          int64
DNI POWSZEDNIE_WEEKEND                             int64
PORA_DNIA                                       category
SPECJALNA_BUDOWA_OBIEKTU                           int64
ILOŚĆ_MIESZKAŃCÓW_OBIEKTU                       category
CZY_BUDYNEK_JEST_NORMALNIE_ZAJĘTY                  int64
CZY_CZUJNIK_DYMU_JEST_W_OBIEKCIE                  object
LICZBA_CZUJNIKÓW_DYMU                              int64
RODZAJ_CZUJNIKA_DYMU_W_MIESZKANIU                 object
POWÓD_ZŁEGO_ZADZIAŁANIA_ALRMU                     object
CZAS_DO_ZAUWAŻENIA_POŻARU                         object
CZAS_MIĘDZY_ODKRYCIEM_A_ZGŁOSZENIEM               object
PÓŹNE_WYWOŁANIE                                    int64
PRZYPADKOWE_CZY_CELOWE                             int64
CO_NAPĘDZA_ZAPŁON                                 object
ŹRÓDŁO_POŻARU                                     object
LOKACJA_ROZPOCZĘCIA_POŻARU     

In [4]:
plt.rcParams["figure.figsize"] = (20,20)

In [5]:
target_name = "PRZYPADKOWE_CZY_CELOWE"
target = dfj[target_name]

data = dfj.drop(columns=[target_name])

In [6]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [7]:
numerical_columns

['ZAG_LUDNOŚCI',
 'GDP_NA_OS',
 'DNI POWSZEDNIE_WEEKEND',
 'SPECJALNA_BUDOWA_OBIEKTU',
 'CZY_BUDYNEK_JEST_NORMALNIE_ZAJĘTY',
 'LICZBA_CZUJNIKÓW_DYMU',
 'PÓŹNE_WYWOŁANIE',
 'CZY_SĄSIEDNI_OBIEKT_ZOSTAŁ_DOTKNIĘTY_POŻAREM',
 'KOD_POJAZDÓW',
 'KOD_PERSONELU',
 'KOD_CZAS_INCYDENTU',
 'POSZKODOWANI_MARTWI',
 'URATOWANI',
 'KOD_EWAKUOWANI',
 'KOD_OBSZAR_USZKODZONY_PRZEZ_POŻAR',
 'KOD_OBSZAR USZKODZONY PRZEZ POŻAR_ORAZ_WODĘ',
 'SĄSIEDNI_OBIEKT_USZKODZONY']

In [8]:
categorical_columns

['PORA_DNIA',
 'ILOŚĆ_MIESZKAŃCÓW_OBIEKTU',
 'CZY_CZUJNIK_DYMU_JEST_W_OBIEKCIE',
 'RODZAJ_CZUJNIKA_DYMU_W_MIESZKANIU',
 'POWÓD_ZŁEGO_ZADZIAŁANIA_ALRMU',
 'CZAS_DO_ZAUWAŻENIA_POŻARU',
 'CZAS_MIĘDZY_ODKRYCIEM_A_ZGŁOSZENIEM',
 'CO_NAPĘDZA_ZAPŁON',
 'ŹRÓDŁO_POŻARU',
 'LOKACJA_ROZPOCZĘCIA_POŻARU',
 'ZASIĘG_OGNIA_PO_PRZYBYCIU',
 'PRZEDMIOT_KTÓRY_ZAPŁONĄŁ',
 'PRZEDMIOT_ROZPRZESTRZENIAJĄCY',
 'SZYBKI_WZROST_POŻARU']

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
minmax =MinMaxScaler()

In [10]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('min_max_scaler', minmax, numerical_columns)])

In [11]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

In [12]:
data_test

,ZAG_LUDNOŚCI,GDP_NA_OS,DNI POWSZEDNIE_WEEKEND,PORA_DNIA,SPECJALNA_BUDOWA_OBIEKTU,ILOŚĆ_MIESZKAŃCÓW_OBIEKTU,CZY_BUDYNEK_JEST_NORMALNIE_ZAJĘTY,CZY_CZUJNIK_DYMU_JEST_W_OBIEKCIE,LICZBA_CZUJNIKÓW_DYMU,RODZAJ_CZUJNIKA_DYMU_W_MIESZKANIU,...,SZYBKI_WZROST_POŻARU,KOD_POJAZDÓW,KOD_PERSONELU,KOD_CZAS_INCYDENTU,POSZKODOWANI_MARTWI,URATOWANI,KOD_EWAKUOWANI,KOD_OBSZAR_USZKODZONY_PRZEZ_POŻAR,KOD_OBSZAR USZKODZONY PRZEZ POŻAR_ORAZ_WODĘ,SĄSIEDNI_OBIEKT_USZKODZONY
185609,709,32358,1,Popołudnie,0,3 lub więcej osób dorosłych poniżej wieku emer...,1,Alarm obecny i podniesiony alarm,1,Zasilanie sieciowe,...,0,1,2,1,0,0,0,0,0,0
56081,681,29354,1,Popołudnie,0,Samotna osoba w wieku emerytalnym,1,Alarm obecny i podniesiony alarm,1,Zasilanie bateryjne,...,0,2,7,2,0,0,0,0,2,0
142215,148,24593,1,Wieczór,0,3 lub więcej osób dorosłych poniżej wieku emer...,1,"Alarm obecny, ale nie działa",1,Zasilanie bateryjne,...,0,2,7,8,0,0,0,1,5,0
302354,5700,55092,1,Popołudnie,0,Samotna osoba w wieku emerytalnym,1,Alarm obecny i podniesiony alarm,1,Zasilanie sieciowe,...,0,2,6,3,0,0,0,1,2,0
143385,316,19367,1,Noc,0,Inne,1,Alarm obecny i podniesiony alarm,1,Zasilanie sieciowe,...,0,4,8,8,0,0,0,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304216,2222,29057,1,Wieczór,0,Samotna osoba poniżej wieku emerytalnego,1,Brak alarmu,0,Brak alarmu,...,1,3,8,2,0,0,0,1,1,0
51738,5218,47812,1,Poranek,0,Samotna osoba w wieku emerytalnym,1,Brak alarmu,0,Brak alarmu,...,0,1,2,3,0,0,0,1,2,0
334415,391,25766,1,Popołudnie,1,Para z dzieckiem/dziećmi,1,Brak alarmu,0,Brak alarmu,...,0,2,7,2,0,0,0,1,1,0
323142,163,24431,0,Popołudnie,0,3 lub więcej osób dorosłych z dzieckiem/dziećmi,1,"Alarm obecny, ale nie podniósł alarmu",1,Zasilanie bateryjne,...,0,3,8,6,0,0,0,1,1,0


# REGRESJA LOGISTYCZNA

In [11]:
from sklearn.linear_model import LogisticRegression


model = make_pipeline(preprocessor, LogisticRegression(n_jobs = -1))

In [12]:
from sklearn import set_config
set_config(display='diagram')
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['PORA_DNIA',
                                                   'ILOŚĆ_MIESZKAŃCÓW_OBIEKTU',
                                                   'CZY_CZUJNIK_DYMU_JEST_W_OBIEKCIE',
                                                   'RODZAJ_CZUJNIKA_DYMU_W_MIESZKANIU',
                                                   'POWÓD_ZŁEGO_ZADZIAŁANIA_ALRMU',
                                                   'CZAS_DO_ZAUWAŻENIA_POŻARU',
                                                   'CZAS_MIĘDZY_ODKRYCIEM_A_ZGŁOSZENIEM',
                                                   'CO_NAPĘDZA_Z...
                                                   'LICZBA_CZUJNIKÓW_DYMU',
                                                   'PÓŹNE_WYWOŁANIE',
                                                   'CZY_SĄSIEDNI_OBIEKT_ZOSTAŁ_DOTKNIĘTY_POŻAREM',
                                                   'KOD_POJAZDÓW',
                                                   'KOD_PERSONELU',
                                                   'KOD_CZAS_INCYDENTU',
                                                   'POSZKODOWANI_MARTWI',
                                                   'URATOWANI',
                                                   'KOD_EWAKUOWANI',
                                                   'KOD_OBSZAR_USZKODZONY_PRZEZ_POŻAR',
                                                   'KOD_OBSZAR USZKODZONY '
                                                   'PRZEZ POŻAR_ORAZ_WODĘ',
                                                   'SĄSIEDNI_OBIEKT_USZKODZONY'])])),
                ('logisticregression', LogisticRegression(n_jobs=-1))])

In [33]:
start = time.time()

In [ ]:
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'logisticregression', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__one-hot-encoder', 'columntransformer__min_max_scaler', 'columntransformer__one-hot-encoder__categories', 'columntransformer__one-hot-encoder__drop', 'columntransformer__one-hot-encoder__dtype', 'columntransformer__one-hot-encoder__handle_unknown', 'columntransformer__one-hot-encoder__sparse', 'columntransformer__min_max_scaler__clip', 'columntransformer__min_max_scaler__copy', 'columntransformer__min_max_scaler__feature_range', 'logisticregression__C', 'logisticregression__class_weight', 'logisticregression__dual', 'logisticregression__fit_intercept', 'logisticregression__intercept_scaling', 'logisticregression__l1_ratio', 'logisticr

In [ ]:
param1 = {}
param1['logisticregression__penalty'] = ['l2']
param1['logisticregression__C'] = [1.0,1.1,2.0,3.0]
param1['logisticregression__intercept_scaling'] = [1.0,1.1,2.0,3.0]
param1['logisticregression__solver'] = ['newton-cg', 'lbfgs']
param1['logisticregression__max_iter'] = [1000,1200]
param1['logisticregression__multi_class'] = ['auto', 'ovr', 'multinomial']
param1

{'logisticregression__max_iter': [1000, 1200],
 'logisticregression__penalty': ['l2']}

In [ ]:
grid1=GridSearchCV(model,param_grid=param1,scoring="accuracy", refit=True,error_score=1)

In [ ]:
model1 = grid1.fit(data_train, target_train)

In [ ]:
grid1.best_score_

0.9445167548659686

In [ ]:
grid1.best_params_

{'logisticregression__max_iter': 1000, 'logisticregression__penalty': 'l2'}

In [ ]:
end = time.time()
time1 =end-start
time1

NameError: ignored

In [ ]:
cv_results1 = pd.DataFrame.from_dict(grid1.cv_results_)
df1 = cv_results1[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df1)

                                              params  mean_test_score
0  {'logisticregression__max_iter': 1000, 'logist...         0.944517
1  {'logisticregression__max_iter': 1200, 'logist...         0.944517


In [ ]:
df1.to_excel("/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/pred1.xlsx")

# Random forest

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

model2 = make_pipeline(preprocessor, RandomForestClassifier())

In [35]:
from sklearn import set_config
set_config(display='diagram')
model2

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['PORA_DNIA',
                                                   'ILOŚĆ_MIESZKAŃCÓW_OBIEKTU',
                                                   'CZY_CZUJNIK_DYMU_JEST_W_OBIEKCIE',
                                                   'RODZAJ_CZUJNIKA_DYMU_W_MIESZKANIU',
                                                   'POWÓD_ZŁEGO_ZADZIAŁANIA_ALRMU',
                                                   'CZAS_DO_ZAUWAŻENIA_POŻARU',
                                                   'CZAS_MIĘDZY_ODKRYCIEM_A_ZGŁOSZENIEM',
                                                   'CO_NAPĘDZA_Z...
                                                   'LICZBA_CZUJNIKÓW_DYMU',
                                                   'PÓŹNE_WYWOŁANIE',
                                                   'CZY_SĄSIEDNI_OBIEKT_ZOSTAŁ_DOTKNIĘTY_POŻAREM',
                                                   'KOD_POJAZDÓW',
                                                   'KOD_PERSONELU',
                                                   'KOD_CZAS_INCYDENTU',
                                                   'POSZKODOWANI_MARTWI',
                                                   'URATOWANI',
                                                   'KOD_EWAKUOWANI',
                                                   'KOD_OBSZAR_USZKODZONY_PRZEZ_POŻAR',
                                                   'KOD_OBSZAR USZKODZONY '
                                                   'PRZEZ POŻAR_ORAZ_WODĘ',
                                                   'SĄSIEDNI_OBIEKT_USZKODZONY'])])),
                ('randomforestclassifier', RandomForestClassifier())])

In [36]:
start = time.time()

In [37]:
model2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'randomforestclassifier', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__one-hot-encoder', 'columntransformer__min_max_scaler', 'columntransformer__one-hot-encoder__categories', 'columntransformer__one-hot-encoder__drop', 'columntransformer__one-hot-encoder__dtype', 'columntransformer__one-hot-encoder__handle_unknown', 'columntransformer__one-hot-encoder__max_categories', 'columntransformer__one-hot-encoder__min_frequency', 'columntransformer__one-hot-encoder__sparse', 'columntransformer__min_max_scaler__clip', 'columntransformer__min_max_scaler__copy', 'columntransformer__min_max_scaler__feature_range', 'randomforestclassifier__bootstrap', 'randomforestclassifier__ccp_alpha', 'randomforestclassifier_

In [38]:
param2 = {}
param2['randomforestclassifier__n_estimators'] = [300,400,500]
param2['randomforestclassifier__criterion'] = ['gini', 'entropy', 'log_loss']
#param2['randomforestclassifier__min_samples_split'] = [2,6,10]
#param2['randomforestclassifier__min_samples_leaf'] = [1,2,6]
param2['randomforestclassifier__bootstrap'] = ['True']
#param2['randomforestclassifier__max_features'] = [None, 'sqrt', 'log2']
param2

{'randomforestclassifier__n_estimators': [100, 200, 300],
 'randomforestclassifier__criterion': ['gini', 'entropy', 'log_loss'],
 'randomforestclassifier__bootstrap': ['True']}

In [39]:
grid2=GridSearchCV(model2,param_grid=param2,scoring="accuracy", n_jobs = -1)

In [40]:
model2 = grid2.fit(data_train, target_train)

In [41]:
grid2.best_score_

0.9487989167488822

In [42]:
grid2.best_params_

{'randomforestclassifier__bootstrap': 'True',
 'randomforestclassifier__criterion': 'gini',
 'randomforestclassifier__n_estimators': 300}

In [43]:
end = time.time()
time2 =end-start
time2

8593.625869989395

In [44]:
scores2 = model2.cv_results_['mean_test_score']
scores2

array([0.94820973, 0.94844079, 0.94879892, 0.94815197, 0.94865643,
       0.94874115, 0.94814042, 0.94879121, 0.94876811])

In [45]:
cv_results2 = pd.DataFrame.from_dict(grid2.cv_results_)
df2 = cv_results2[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df2)

                                              params  mean_test_score
0  {'randomforestclassifier__bootstrap': 'True', ...         0.948210
1  {'randomforestclassifier__bootstrap': 'True', ...         0.948441
2  {'randomforestclassifier__bootstrap': 'True', ...         0.948799
3  {'randomforestclassifier__bootstrap': 'True', ...         0.948152
4  {'randomforestclassifier__bootstrap': 'True', ...         0.948656
5  {'randomforestclassifier__bootstrap': 'True', ...         0.948741
6  {'randomforestclassifier__bootstrap': 'True', ...         0.948140
7  {'randomforestclassifier__bootstrap': 'True', ...         0.948791
8  {'randomforestclassifier__bootstrap': 'True', ...         0.948768


In [46]:
df2.to_excel("pred2.xlsx")

# Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
param3 = {}
param3['decisiontreeclassifier__criterion'] = ['gini', 'entropy']
param3['decisiontreeclassifier__max_features'] = [None, 'sqrt', 'log2']
param3['decisiontreeclassifier__splitter'] = ['best', 'random']
param3['decisiontreeclassifier__min_samples_split'] = [2, 6, 10]
param3['decisiontreeclassifier__max_features'] = ['sqrt', 'log2']
param3

{'decisiontreeclassifier__criterion': ['gini', 'entropy'],
 'decisiontreeclassifier__max_features': ['sqrt', 'log2'],
 'decisiontreeclassifier__min_samples_split': [2, 6, 10],
 'decisiontreeclassifier__splitter': ['best', 'random']}

In [ ]:
model3 = make_pipeline(preprocessor, DecisionTreeClassifier())

In [ ]:
grid3=GridSearchCV(model3,param_grid=param3,scoring="accuracy", n_jobs = n_cpus)

In [ ]:
model3 = grid3.fit(data_train, target_train)

In [ ]:
model3.best_score_

In [ ]:
model3.best_params_

In [ ]:
model3.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'decisiontreeclassifier', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__one-hot-encoder', 'columntransformer__min_max_scaler', 'columntransformer__one-hot-encoder__categories', 'columntransformer__one-hot-encoder__drop', 'columntransformer__one-hot-encoder__dtype', 'columntransformer__one-hot-encoder__handle_unknown', 'columntransformer__one-hot-encoder__sparse', 'columntransformer__min_max_scaler__clip', 'columntransformer__min_max_scaler__copy', 'columntransformer__min_max_scaler__feature_range', 'decisiontreeclassifier__ccp_alpha', 'decisiontreeclassifier__class_weight', 'decisiontreeclassifier__criterion', 'decisiontreeclassifier__max_depth', 'decisiontreeclassifier__max_features', 'decisiontreec

In [ ]:
end = time.time()
time3 =end-start
time3

In [ ]:
from sklearn.tree import plot_tree
from sklearn import tree

In [ ]:
cv_results3 = pd.DataFrame.from_dict(grid3.cv_results_)
df3 = cv_results3[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df3)

In [ ]:
df3.to_excel("/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/pred3.xlsx")

# K NAJBLIŻSZYCH SĄSIADÓW

In [34]:
from sklearn.neighbors import KNeighborsClassifier

In [35]:
start = time.time()

In [36]:
param4 = {}
param4['kneighborsclassifier__n_neighbors'] = [6,7,8]
#param4['kneighborsclassifier__weights'] = ['uniform', 'distance']
#param4['kneighborsclassifier__leaf_size'] = [20, 30]
#param4['kneighborsclassifier__p'] = [1,2]
param4

{'kneighborsclassifier__n_neighbors': [6, 7, 8]}

In [37]:
model4 = make_pipeline(preprocessor, KNeighborsClassifier(n_jobs = 10))

In [38]:
grid4=GridSearchCV(model4,param_grid=param4,scoring="accuracy", refit=True, n_jobs = 10)

In [39]:
model4 = grid4.fit(data_train, target_train)

In [40]:
model4.best_score_

0.9330835408694768

In [41]:
model4.best_params_

{'kneighborsclassifier__n_neighbors': 7}

In [42]:
model4.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__columntransformer', 'estimator__kneighborsclassifier', 'estimator__columntransformer__n_jobs', 'estimator__columntransformer__remainder', 'estimator__columntransformer__sparse_threshold', 'estimator__columntransformer__transformer_weights', 'estimator__columntransformer__transformers', 'estimator__columntransformer__verbose', 'estimator__columntransformer__verbose_feature_names_out', 'estimator__columntransformer__one-hot-encoder', 'estimator__columntransformer__min_max_scaler', 'estimator__columntransformer__one-hot-encoder__categories', 'estimator__columntransformer__one-hot-encoder__drop', 'estimator__columntransformer__one-hot-encoder__dtype', 'estimator__columntransformer__one-hot-encoder__handle_unknown', 'estimator__columntransformer__one-hot-encoder__max_categories', 'estimator__columntransformer__one-hot-encoder__min_frequency', 'estimator__columntransformer__one-hot-enco

In [43]:
end = time.time()
time4 =end-start
time4

4839.557332515717

In [44]:
cv_results4 = pd.DataFrame.from_dict(grid4.cv_results_)
df4 = cv_results4[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df4)

                                     params  mean_test_score
0  {'kneighborsclassifier__n_neighbors': 6}         0.929918
1  {'kneighborsclassifier__n_neighbors': 7}         0.933084
2  {'kneighborsclassifier__n_neighbors': 8}         0.931909


In [45]:
df4.to_excel("pred4.1.xlsx")

# SVC

In [ ]:
from sklearn.svm import SVC

In [ ]:
model5 = make_pipeline(preprocessor, SVC())

In [ ]:
start = time.time()

In [ ]:
param5 = {}
#param5['svc__C'] = [1.0,2.0]
param5['svc__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid','precomputed']
#param5['svc__degree'] = [3, 5]
#param5['svc__gamma'] = ['scale', 'auto']
#param5['svc__decision_function_shape'] = ['ovo', 'ovr']
param5

In [ ]:
grid5=GridSearchCV(model5,param_grid=param5,scoring="accuracy", refit=True, n_jobs = -1)

In [ ]:
model5 = grid5.fit(data_train, target_train)

In [ ]:
model5.best_score_

In [ ]:
model5.best_params_

In [ ]:
model5.get_params().keys()

In [ ]:
end = time.time()
time5 =end-start
time5

In [ ]:
cv_results5 = pd.DataFrame.from_dict(grid5.cv_results_)
df5 = cv_results5[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df5)

In [ ]:
df5.to_excel("pred5.xlsx")

# EXTRA TREE CLASSIFIER

In [ ]:
from sklearn.tree import ExtraTreeClassifier

In [ ]:
model6 = make_pipeline(preprocessor, ExtraTreeClassifier())

In [ ]:
start = time.time()

In [ ]:
param6 = {}
param6['extratreeclassifier__criterion'] = ['gini', 'entropy','log_loss']
param6['extratreeclassifier__splitter'] = ['random', 'best']
param6['extratreeclassifier__max_features'] = ['auto', 'sqrt', 'log2']
param6['extratreeclassifier__min_samples_split'] = [2, 4, 6]
param6['extratreeclassifier__min_samples_leaf'] = [1, 2, 4]
param6['extratreeclassifier__max_features'] = ['sqrt', 'log2', None]
param6

{'extratreeclassifier__criterion': ['gini', 'entropy', 'log_loss'],
 'extratreeclassifier__max_features': ['sqrt', 'log2', None],
 'extratreeclassifier__min_samples_leaf': [1, 2, 4],
 'extratreeclassifier__min_samples_split': [2, 4, 6],
 'extratreeclassifier__splitter': ['random', 'best']}

In [ ]:
grid6=GridSearchCV(model6,param_grid=param6,scoring="accuracy",refit=True, n_jobs = n_cpus)

In [ ]:
model6 = grid6.fit(data_train, target_train)

In [ ]:
model6.best_score_

In [ ]:
model6.best_params_

In [ ]:
model6.get_params().keys()

In [ ]:
end = time.time()
time6 =end-start
time6

In [ ]:
cv_results6 = pd.DataFrame.from_dict(grid6.cv_results_)
df6 = cv_results6[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df6)

In [ ]:
df6.to_excel("/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/pred6.xlsx")

# DUMMY CLASSIFIER

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
model7 = make_pipeline(preprocessor, DummyClassifier())

In [ ]:
start = time.time()

In [ ]:
model7 = model7.fit(data_train, target_train)

In [ ]:
y_pred7=model7.predict(data_test)
model7.predict(data_test)[:10]

In [ ]:
target_test[:10]

In [ ]:
model7.score(data_test, target_test)

In [ ]:
end = time.time()
time7 =end-start
time7

In [ ]:
cv_results7 = pd.DataFrame.from_dict(grid7.cv_results_)
df7 = cv_results7[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df7)

In [ ]:
df7.to_excel("/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/pred7.xlsx")

# CAT BOOST

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model8 = make_pipeline(preprocessor, CatBoostClassifier())

In [ ]:
start = time.time()

In [ ]:
param8 = {}
param8['catboostclassifier__iterations'] = [100,200,500,1000]
param8['catboostclassifier__depth'] = [4, 6, 8]
param8

{'catboostclassifier__depth': [4, 6, 8],
 'catboostclassifier__iterations': [100, 200, 500, 1000]}

In [ ]:
grid8=GridSearchCV(model8,param_grid=param8,scoring="accuracy",refit=True, n_jobs = n_cpus)

In [ ]:
model8 = grid8.fit(data_train, target_train)

Learning rate set to 0.110637
0:	learn: 0.5048913	total: 133ms	remaining: 2m 12s
1:	learn: 0.3835838	total: 210ms	remaining: 1m 44s
2:	learn: 0.3035893	total: 289ms	remaining: 1m 36s
3:	learn: 0.2519025	total: 368ms	remaining: 1m 31s
4:	learn: 0.2192914	total: 448ms	remaining: 1m 29s
5:	learn: 0.1977562	total: 523ms	remaining: 1m 26s
6:	learn: 0.1832443	total: 614ms	remaining: 1m 27s
7:	learn: 0.1732089	total: 695ms	remaining: 1m 26s
8:	learn: 0.1678664	total: 778ms	remaining: 1m 25s
9:	learn: 0.1616971	total: 857ms	remaining: 1m 24s
10:	learn: 0.1581584	total: 937ms	remaining: 1m 24s
11:	learn: 0.1556823	total: 1.01s	remaining: 1m 23s
12:	learn: 0.1530973	total: 1.09s	remaining: 1m 23s
13:	learn: 0.1505895	total: 1.17s	remaining: 1m 22s
14:	learn: 0.1481137	total: 1.25s	remaining: 1m 22s
15:	learn: 0.1462584	total: 1.34s	remaining: 1m 22s
16:	learn: 0.1451518	total: 1.42s	remaining: 1m 21s
17:	learn: 0.1437124	total: 1.49s	remaining: 1m 21s
18:	learn: 0.1425476	total: 1.58s	remaining:

In [ ]:
model8.best_score_

0.950366220133566

In [ ]:
model8.best_params_

{'catboostclassifier__depth': 8, 'catboostclassifier__iterations': 1000}

In [ ]:
model8.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__columntransformer', 'estimator__catboostclassifier', 'estimator__columntransformer__n_jobs', 'estimator__columntransformer__remainder', 'estimator__columntransformer__sparse_threshold', 'estimator__columntransformer__transformer_weights', 'estimator__columntransformer__transformers', 'estimator__columntransformer__verbose', 'estimator__columntransformer__verbose_feature_names_out', 'estimator__columntransformer__one-hot-encoder', 'estimator__columntransformer__min_max_scaler', 'estimator__columntransformer__one-hot-encoder__categories', 'estimator__columntransformer__one-hot-encoder__drop', 'estimator__columntransformer__one-hot-encoder__dtype', 'estimator__columntransformer__one-hot-encoder__handle_unknown', 'estimator__columntransformer__one-hot-encoder__sparse', 'estimator__columntransformer__min_max_scaler__clip', 'estimator__columntransformer__min_max_scaler__copy', 'estimato

In [ ]:
end = time.time()
time8 =end-start
time8

1976.925044298172

In [ ]:
cv_results8 = pd.DataFrame.from_dict(grid8.cv_results_)
df8 = cv_results8[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df8)

                                               params  mean_test_score
0   {'catboostclassifier__depth': 4, 'catboostclas...         0.948895
1   {'catboostclassifier__depth': 4, 'catboostclas...         0.949635
2   {'catboostclassifier__depth': 4, 'catboostclas...         0.950178
3   {'catboostclassifier__depth': 4, 'catboostclas...         0.950231
4   {'catboostclassifier__depth': 6, 'catboostclas...         0.949319
5   {'catboostclassifier__depth': 6, 'catboostclas...         0.949153
6   {'catboostclassifier__depth': 6, 'catboostclas...         0.950127
7   {'catboostclassifier__depth': 6, 'catboostclas...         0.950343
8   {'catboostclassifier__depth': 8, 'catboostclas...         0.948206
9   {'catboostclassifier__depth': 8, 'catboostclas...         0.947863
10  {'catboostclassifier__depth': 8, 'catboostclas...         0.949758
11  {'catboostclassifier__depth': 8, 'catboostclas...         0.950366


In [ ]:
df8.to_excel("/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/pred8.xlsx")

# LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
model9 = make_pipeline(preprocessor, LGBMClassifier(n_jobs = n_cpus))

In [ ]:
start = time.time()

In [ ]:
param9 = {}
param9['lgbmclassifier__boosting_type'] = ['gbdt', 'dart', 'goss']
param9['lgbmclassifier__num_leaves'] = [31,35,40]
param9['lgbmclassifier__n_estimators'] = [100,200,300]
param9

{'lgbmclassifier__boosting_type': ['gbdt', 'dart', 'goss'],
 'lgbmclassifier__n_estimators': [100, 200, 300],
 'lgbmclassifier__num_leaves': [31, 35, 40]}

In [ ]:
grid9=GridSearchCV(model9,param_grid=param9,scoring="accuracy",refit=True, n_jobs = n_cpus)

In [ ]:
model9 = grid9.fit(data_train, target_train)

In [ ]:
model9 = model9.fit(data_train, target_train)

In [ ]:
model9.best_score_

0.9495228804831571

In [ ]:
model9.best_params_

{'lgbmclassifier__boosting_type': 'gbdt',
 'lgbmclassifier__n_estimators': 300,
 'lgbmclassifier__num_leaves': 35}

In [ ]:
model9.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__columntransformer', 'estimator__lgbmclassifier', 'estimator__columntransformer__n_jobs', 'estimator__columntransformer__remainder', 'estimator__columntransformer__sparse_threshold', 'estimator__columntransformer__transformer_weights', 'estimator__columntransformer__transformers', 'estimator__columntransformer__verbose', 'estimator__columntransformer__verbose_feature_names_out', 'estimator__columntransformer__one-hot-encoder', 'estimator__columntransformer__min_max_scaler', 'estimator__columntransformer__one-hot-encoder__categories', 'estimator__columntransformer__one-hot-encoder__drop', 'estimator__columntransformer__one-hot-encoder__dtype', 'estimator__columntransformer__one-hot-encoder__handle_unknown', 'estimator__columntransformer__one-hot-encoder__sparse', 'estimator__columntransformer__min_max_scaler__clip', 'estimator__columntransformer__min_max_scaler__copy', 'estimator__c

In [ ]:
end = time.time()
time9 =end-start
time9

16161.823615789413

In [ ]:
cv_results9 = pd.DataFrame.from_dict(grid9.cv_results_)
df9 = cv_results9[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df9)

                                               params  mean_test_score
0   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949099
1   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.948837
2   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949084
3   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949134
4   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949180
5   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949454
6   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949504
7   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949523
8   {'lgbmclassifier__boosting_type': 'gbdt', 'lgb...         0.949519
9   {'lgbmclassifier__boosting_type': 'dart', 'lgb...         0.946835
10  {'lgbmclassifier__boosting_type': 'dart', 'lgb...         0.947189
11  {'lgbmclassifier__boosting_type': 'dart', 'lgb...         0.947428
12  {'lgbmclassifier__boosting_type': 'dart', 'lgb...         0.948383
13  {'

In [ ]:
df9.to_excel("/content/drive/MyDrive/Organizacja/Nauka/MGR IiE/Python/pred9.xlsx")

# MLPClassifier

In [46]:
from sklearn.neural_network import MLPClassifier

In [47]:
model10 = make_pipeline(preprocessor, MLPClassifier())

In [48]:
start = time.time()

In [49]:
param10 = {}
param10['mlpclassifier__max_iter'] = [500, 750, 1000]
param10['mlpclassifier__activation'] = ['identity', 'logistic', 'tanh', 'relu']
param10

{'mlpclassifier__max_iter': [500, 750, 1000],
 'mlpclassifier__activation': ['identity', 'logistic', 'tanh', 'relu']}

In [50]:
model10.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'mlpclassifier', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__one-hot-encoder', 'columntransformer__min_max_scaler', 'columntransformer__one-hot-encoder__categories', 'columntransformer__one-hot-encoder__drop', 'columntransformer__one-hot-encoder__dtype', 'columntransformer__one-hot-encoder__handle_unknown', 'columntransformer__one-hot-encoder__max_categories', 'columntransformer__one-hot-encoder__min_frequency', 'columntransformer__one-hot-encoder__sparse', 'columntransformer__min_max_scaler__clip', 'columntransformer__min_max_scaler__copy', 'columntransformer__min_max_scaler__feature_range', 'mlpclassifier__activation', 'mlpclassifier__alpha', 'mlpclassifier__batch_size', 'mlpclassifier__beta_1', 

In [51]:
grid10=GridSearchCV(model10,param_grid=param10,scoring="accuracy",refit=True, n_jobs = -1)

In [52]:
model10 = grid10.fit(data_train, target_train)

In [53]:
model10.best_score_

0.9444281853556381

In [54]:
model10.best_params_

{'mlpclassifier__activation': 'identity', 'mlpclassifier__max_iter': 500}

In [55]:
end = time.time()
time10 =end-start
time10

4634.79953289032

In [56]:
cv_results10 = pd.DataFrame.from_dict(grid10.cv_results_)
df10 = cv_results10[['params', 'mean_test_score']]

pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('expand_frame_repr', False)
print(df10)

                                               params  mean_test_score
0   {'mlpclassifier__activation': 'identity', 'mlp...         0.944428
1   {'mlpclassifier__activation': 'identity', 'mlp...         0.944359
2   {'mlpclassifier__activation': 'identity', 'mlp...         0.944313
3   {'mlpclassifier__activation': 'logistic', 'mlp...         0.938871
4   {'mlpclassifier__activation': 'logistic', 'mlp...         0.938648
5   {'mlpclassifier__activation': 'logistic', 'mlp...         0.939799
6   {'mlpclassifier__activation': 'tanh', 'mlpclas...         0.937666
7   {'mlpclassifier__activation': 'tanh', 'mlpclas...         0.938482
8   {'mlpclassifier__activation': 'tanh', 'mlpclas...         0.938386
9   {'mlpclassifier__activation': 'relu', 'mlpclas...         0.939326
10  {'mlpclassifier__activation': 'relu', 'mlpclas...         0.938648
11  {'mlpclassifier__activation': 'relu', 'mlpclas...         0.938752


In [57]:
df10.to_excel("pred10.xlsx")